In [30]:
import cirq

# Define two qubits
q0 = cirq.LineQubit(0)
q1 = cirq.LineQubit(1)

# Define the |01> state
initial_state = [0, 1, 0, 0]  # |01>

# Define the XX + YY operator using Cirq's PauliString
xx = cirq.PauliString(cirq.X(q0) * cirq.X(q1))
yy = cirq.PauliString(cirq.Y(q0) * cirq.Y(q1))

# Combine XX + YY
operator = xx + yy

# Create a circuit to apply the operator
circuit = cirq.Circuit()

# Prepare the initial state |01> using single-qubit gates
circuit.append(cirq.X(q1))  # Flip q0 to |1> (now |01>)

circuit.append(operator)

## Add the XX + YY operator as a gate
#circuit.append(cirq.PauliSumExponential(operator, exponent=1.0))
#
# Simulate the circuit
simulator = cirq.Simulator()
result = simulator.simulate(circuit, initial_state=initial_state)

# Print the final state vector
print("Final state vector after applying XX + YY:")
print(result.final_state_vector)
#

Final state vector after applying XX + YY:
[-1.+0.j  0.+0.j  0.+0.j  0.+0.j]


In [31]:
circuit

0: ───────X─────────────────Y─────────────────
          │                 │
1: ───X───PauliString(+X)───PauliString(+Y)───